# Path Optimization (KOMO)



## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../test/kitchen.g')

Let's add some objects

In [2]:
C.addObject(name='item1', parent='sink1', shape=ry.ST.ssBox, pos=[-.1, -.1, .52], size=[.1, .1, .25, .02], color=[1., 0., 0.])
C.addObject('item2', 'sink1', ry.ST.ssBox, [.1, .1, .25, .02], [1., 1., 0.], [.1, .1, .52])
C.addObject('tray', 'stove1', ry.ST.ssBox, [.2, .2, .05, .02], [0., 1., 0.], [.0, .0, .42])

compute a collision free path to touch object item1 with pr2L:

In [3]:
X0 = C.getFrameState()

obj1 = "item1";
arm = "pr2R";

komo = C.komo_path(1.,20, 10., True);

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.distance, [arm, obj1], ry.OT.eq, [1e2]);
komo.addObjective([.9,1.], ry.FS.positionDiff, ["endeffWorkspace", obj1], ry.OT.sos, [1e0]);
komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);

komo.optimize(True)
komo.getReport()

[{'x_dim': 500, 'T': 20, 'k_order': 2, 'tau': 0.5, 'useSwift': True},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.0},
 {'order': 2.0,
  'type': 'sos',
  'feature': 'qItself#46',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 6.409665161412653},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'qItself#46',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.001021876683872976},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'ProxyCost',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.0},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'qLimits',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'inEq_sumOfPos': 0.3481930780352092},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'PairCollision-pr2R-item1',
  'vars': [0, 0, 0, 0, 0, 0, 0, 

In [5]:
#komo.displayTrajectory()
V = komo.view()

In [6]:
V=0

Best way to read out: grab a configuration into K and analyze it here:

In [7]:
C.setFrameState( komo.getConfiguration(19) )
C.getJointState()

array([-0.50090023,  0.90329872,  1.81616592,  0.10001433,  0.00235007,
       -0.00229488, -0.3344213 ,  1.00220512,  0.39371048,  0.25781461,
        0.50156163, -0.95573083,  1.00604442, -0.7676919 , -1.92485412,
       -1.72243683,  1.49740078, -0.28967774, -0.49316179, -0.50055609,
        0.50192907,  0.09686037,  0.09520579,  0.01223921,  0.00356277])